#ラデックの特徴量追加#27
https://github.com/users/coffeemountain/projects/5/views/1?layout=board
* ラデックコードにありは対象外

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

In [3]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/train_valid_validlabel/otto-train-and-test-data-for-local-validation/'

In [4]:
df_val = pd.read_parquet(INPUT_DIR+'test.parquet')

df_val

,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0
...,...,...,...,...
7683572,12899774,33035,1661723968,0
7683573,12899775,1743151,1661723970,0
7683574,12899776,548599,1661723972,0
7683575,12899777,384045,1661723976,0


In [5]:
%%time
# action_num_reverse_chrono_unique 0.2716890769703452　：セッション内のユニークaidごとの順番。

#もっといい書きかたあると思う
def add_action_num_reverse_chrono_unique(df):
  aids = df.aid.to_list()
  unique_aids = df.aid.unique()
  unique_aids_dict = {unique_aids[i]:(i + 1)  for i in range(len(unique_aids))}
  output = [unique_aids_dict[aid] for aid in aids]
  return output

add_action_num_reverse_chrono_unique_df = df_val.groupby('session').apply(lambda x: add_action_num_reverse_chrono_unique(x)).explode()
add_action_num_reverse_chrono_unique_df.name = 'add_action_num_reverse_chrono_unique'
add_action_num_reverse_chrono_unique_df = add_action_num_reverse_chrono_unique_df.to_frame().reset_index()
add_action_num_reverse_chrono_unique_df

CPU times: user 3min 34s, sys: 16 s, total: 3min 50s
Wall time: 3min 28s


,session,add_action_num_reverse_chrono_unique
0,11098528,1
1,11098529,1
2,11098530,1
3,11098530,1
4,11098530,2
...,...,...
7683572,12899774,1
7683573,12899775,1
7683574,12899776,1
7683575,12899777,1


In [17]:
%%time
#covisit_clicks_candidate_num 0.24848206202059703　：covis click candidateの数？
#covisit_buys_candidate_num 0.009287900880354305　：covis click candidateの数。

import itertools
from collections import Counter


#### covis 読み込み######
#click covis
COVIS_MATRIX_PATH = '/content/drive/MyDrive/kaggle/2022/OTTO/input/train_covistation_matrix/chris_baseline/'
VER = 6
DISK_PIECES = 4
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()
top_20_clicks = pqt_to_dict( pd.read_parquet(COVIS_MATRIX_PATH+f'top_20_clicks_v{VER}_0.pqt') )
for k in range(1,DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(COVIS_MATRIX_PATH+f'top_20_clicks_v{VER}_{k}.pqt') ) )
#crart,order covis
#VER = 6
#DISK_PIECES = 4
#def pqt_to_dict(df):
#    return df.groupby('aid_x').aid_y.apply(list).to_dict()
#top_15_carts_orders = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_15_carts_orders_v{VER}_0.pqt') )
#for k in range(1,DISK_PIECES): 
#    top_15_carts_orders.update( pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/output/top_15_carts_orders_v{VER}_{k}.pqt') ) )
top_20_buy2buy = pqt_to_dict( pd.read_parquet(COVIS_MATRIX_PATH + f'top_15_buy2buy_v{VER}_0.pqt') )
#######################

class CFG:
  candidates_count = 50

def covisit_types_candidate_num(df, covis_matrix, k=CFG.candidates_count):
  aids=df.aid.tolist()
  unique_aids = df.aid.unique()
  output = [len(covis_matrix[aid]) for aid in unique_aids if aid in covis_matrix]
  return output

#covisit_clicks_candidate_num
covisit_clicks_candidate_num_df = df_val.groupby('session').apply(lambda x: covisit_types_candidate_num(x, top_20_clicks)).explode()
covisit_clicks_candidate_num_df.name = 'covisit_clicks_candidate_num'
covisit_clicks_candidate_num_df = covisit_clicks_candidate_num_df.to_frame().reset_index()
covisit_clicks_candidate_num_df

#covisit_buys_candidate_num
covisit_buys_candidate_num_df = df_val.groupby('session').apply(lambda x: covisit_types_candidate_num(x, top_20_buy2buy)).explode()
covisit_buys_candidate_num_df.name = 'covisit_buys_candidate_num'
covisit_buys_candidate_num_df = covisit_buys_candidate_num_df.to_frame().reset_index()
covisit_buys_candidate_num_df

CPU times: user 8min 22s, sys: 31.7 s, total: 8min 54s
Wall time: 8min 12s


,session,covisit_buys_candidate_num
0,11098528,15
1,11098529,15
2,11098530,15
3,11098530,15
4,11098531,15
...,...,...
5188679,12899774,15
5188680,12899775,15
5188681,12899776,15
5188682,12899777,15


In [18]:
covisit_buys_candidate_num_df[:40]

,session,covisit_buys_candidate_num
0,11098528,15
1,11098529,15
2,11098530,15
3,11098530,15
4,11098531,15
5,11098531,9
6,11098531,15
7,11098531,15
8,11098531,10
9,11098531,15


In [7]:
%%time
#sec_to_session_end 0.19485467138278798　：セッション終了からの時間

def sec_to_session_end(df):
  tss = df.ts.to_list()
  last_ts = max(tss)
  diff_tss = [(last_ts - ts) for ts in tss]
  return diff_tss

sec_to_session_end_df = df_val.groupby('session').apply(lambda x: sec_to_session_end(x)).explode()
sec_to_session_end_df.name = 'sec_to_session_end_df'
sec_to_session_end_df = sec_to_session_end_df.to_frame().reset_index()
sec_to_session_end_df

CPU times: user 1min 21s, sys: 664 ms, total: 1min 21s
Wall time: 1min 21s


,session,sec_to_session_end_df
0,11098528,0
1,11098529,0
2,11098530,1332
3,11098530,1244
4,11098530,1163
...,...,...
7683572,12899774,0
7683573,12899775,0
7683574,12899776,0
7683575,12899777,0


In [8]:
%%time
#session_length_unique 0.022921177730713577　：セッション内のユニークaidの数？

def session_length_unique(df):
  unique_aids = df.aid.unique()
  return len(unique_aids)

session_length_unique_df = df_val.groupby('session').apply(lambda x: session_length_unique(x)).explode()
session_length_unique_df.name = 'session_length_unique'
session_length_unique_df = session_length_unique_df.to_frame().reset_index()
session_length_unique_df

CPU times: user 2min 23s, sys: 9.97 s, total: 2min 33s
Wall time: 2min 22s


,session,session_length_unique
0,11098528,1
1,11098529,1
2,11098530,2
3,11098531,11
4,11098532,2
...,...,...
1801246,12899774,1
1801247,12899775,1
1801248,12899776,1
1801249,12899777,1


In [9]:
#aid_interacted_with_count 0.004570131040186471　：相互のaidカウント？covis matrixとは別？

# どゆこと？？？

In [10]:
%%time
#aid_clicks_count 0.0016809643174538038　：typeされたaidの数。user観点？
#aid_carted_count 0.00021963718332129753　：カート入れたaidの数。user観点？
#aid_ordered_count 0.0　：オーダ入れたaidの数。user観点？

def aid_type_count(df, types):
  type_count = df[df['type']==types]
  return len(type_count)

types = 0 #clicks
aid_type_count_df = df_val.groupby('session').apply(lambda x: aid_type_count(x, types)).explode()
aid_type_count_df.name = 'aid_type_count'
aid_type_count_df = aid_type_count_df.to_frame().reset_index()
aid_type_count_df

CPU times: user 11min 36s, sys: 29.1 s, total: 12min 5s
Wall time: 11min 15s


,session,aid_type_count
0,11098528,1
1,11098529,1
2,11098530,5
3,11098531,20
4,11098532,2
...,...,...
1801246,12899774,1
1801247,12899775,1
1801248,12899776,1
1801249,12899777,1


In [11]:
%%time
#sec_since_session_start 0.0010450201166368961　：セッション開始時間からの経過時間

def sec_since_session_start(df):
  tss = df.ts.to_list()
  start_ts = min(tss)
  diff_tss = [(ts - start_ts) for ts in tss]
  return diff_tss
    
sec_since_session_start_df = df_val.groupby('session').apply(lambda x: sec_since_session_start(x)).explode()
sec_since_session_start_df.name = 'sec_since_session_start'
sec_since_session_start_df = sec_since_session_start_df.to_frame().reset_index()
sec_since_session_start_df

CPU times: user 1min 20s, sys: 314 ms, total: 1min 21s
Wall time: 1min 20s


,session,sec_since_session_start
0,11098528,0
1,11098529,0
2,11098530,0
3,11098530,88
4,11098530,169
...,...,...
7683572,12899774,0
7683573,12899775,0
7683574,12899776,0
7683575,12899777,0


In [12]:
#this_aid_carted_count 0.0009320732094743725　：このaidがカートされた数。セッション内で重複aidもあるので。
#this_aid_clicked_count 0.0　：このaidがクリックされた数。セッション内で重複aidもあるので。
#this_aid_ordered_count 0.0　：このaidがオーダされた数。セッション内で重複aidもあるので。

#クリスの特徴量？
# ユーザーはこの項目を複数回クリックしましたか? 幾つか →step4 suzuki

In [13]:
%%time
#relative_position_in_session 0.00021918611998695227　：セッション内のポジション。

def relative_position_in_session(df):
  aids = df.aid.to_list()
  relative_positions = [(idx/len(aids)) for idx in range(len(aids))]
  return relative_positions

relative_position_in_session_df = df_val.groupby('session').apply(lambda x: relative_position_in_session(x)).explode()
relative_position_in_session_df.name = 'relative_position_in_session'
relative_position_in_session_df = relative_position_in_session_df.to_frame().reset_index()
relative_position_in_session_df

CPU times: user 1min 20s, sys: 366 ms, total: 1min 21s
Wall time: 1min 21s


,session,relative_position_in_session
0,11098528,0.0
1,11098529,0.0
2,11098530,0.0
3,11098530,0.166667
4,11098530,0.333333
...,...,...
7683572,12899774,0.0
7683573,12899775,0.0
7683574,12899776,0.0
7683575,12899777,0.0


In [14]:
#popularity_0 0.0　：人気aid
#popularity_1 0.0　：人気aid ver2？
#popularity_2 0.0　：人気aid ver3？

#クリスbaselineにある